In [2]:
import findspark
findspark.init('/home/fede/spark-2.1.0-bin-hadoop2.7')

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('logic_regre').getOrCreate()

In [6]:
from pyspark.ml.classification import LogisticRegression

In [7]:
file_path = '/home/fede/sample_libsvm_data.txt'
my_data = spark.read.format('libsvm').load(file_path)

In [9]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [12]:
my_log_reg_model= LogisticRegression(featuresCol='features',labelCol='label')

In [13]:
fitted_logreg = my_log_reg_model.fit(my_data)

In [14]:
log_summary = fitted_logreg.summary

In [16]:
log_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)



In [17]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111056...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126979...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170303...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200604...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...|[1.47109814695581...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102631...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215613...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

In [18]:
lr_train,lr_test = my_data.randomSplit([0.7,0.3])

In [19]:
final_model= LogisticRegression(featuresCol='features',labelCol='label')

In [20]:
fit_final = final_model.fit(lr_train)

In [21]:
prediction_and_labels = fit_final.evaluate(lr_test)

In [23]:
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[22.2687751980841...|[0.99999999978679...|       0.0|
|  0.0|(692,[122,123,124...|[13.0199932110734...|[0.99999778441798...|       0.0|
|  0.0|(692,[122,123,148...|[22.4179168427262...|[0.99999999981633...|       0.0|
|  0.0|(692,[123,124,125...|[28.4737452976040...|[0.99999999999956...|       0.0|
|  0.0|(692,[124,125,126...|[30.3385215283430...|[0.99999999999993...|       0.0|
|  0.0|(692,[126,127,128...|[13.7291850978607...|[0.99999890984011...|       0.0|
|  0.0|(692,[126,127,128...|[19.3164195751290...|[0.99999999591694...|       0.0|
|  0.0|(692,[150,151,152...|[18.3711223653947...|[0.99999998949193...|       0.0|
|  0.0|(692,[152,153,154...|[28.8878321912029...|[0.99999999999971...|       0.0|
|  0.0|(692,[152

In [27]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator,
                                   MulticlassClassificationEvaluator)

In [28]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol = 'rawPrediction',labelCol='label')

In [29]:
my_final_roc = my_eval.evaluate(prediction_and_labels.predictions)

In [30]:
my_final_roc

1.0